In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import os

import pickle

from tqdm import tqdm

import lightgbm
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier 

import catboost
import gc
import random
random.seed(2233)

# for ens X
from sklearn.metrics import average_precision_score 
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib.pyplot as plt

In [2]:
MODEL = "catboost"
all_cols = ['target','AccAP_std_past_10_maxs', 'AccV_mean_past_20_mins', 'AccML_mean_past_10_mins', 'fence_AccAP_pct_m5_w10_past', 'precent_prograss_AccAP', 'AccV_max_past_20_maxs', 'AccAP_std_past_10_mins', 'AccML_mean_past_20_maxs', 'fence_AccAP_pct_m25_w10_past', 'precent_prograss_AccV', 'graph_second_half_AccV_mean', 'AccV_min_past_10_maxs', 'precent_prograss_moving_std_all_AccV', 'AccML_std_past_20_mins', 'AccML_std_past_10_mins', 'graph_second_half_AccV_min', 'AccV_std_past_20_maxs', 'graph_first_half_AccV_std', 'AccV_min_past_20_mins', 'AccV_min_past_20_maxs', 'AccAP_pct', 'AccML_std_past_20_maxs', 'AccAP_min_past_20_maxs', 'AccV_mean_past_10_mins', 'graph_second_half_AccV_max', 'moving_std_AccV_pct_10', 'graph_AccV_min', 'precent_prograss_moving_mean_all_AccAP', 'AccV_max_past_20_mins', 'graph_first_half_AccV_min', 'fence_AccAP_pct_m10_w10_past', 'precent_prograss_moving_mean_all_AccV', 'fence_AccAP_pct_m25_w5_past', 'fence_AccAP_pct_m10_w5_past', 'AccV_max_past_10_maxs', 'graph_second_half_AccV_std', 'AccV_max_past_10_mins', 'AccML_mean_past_20_mins', 'AccML_std_past_10_maxs', 'moving_std_all_AccAP', 'graph_first_half_AccV_mean', 'Subject', 'AccV_std_past_10_maxs', 'AccML_min_past_10_maxs', 'AccAP_std_past_20_maxs', 'graph_first_half_AccV_max', 'AccV_min_past_10_mins', 'moving_std_AccV_pct_5', 'graph_AccV_max', 'precent_prograss_moving_std_all_AccAP', 'AccAP_min_past_10_maxs', 'fence_AccAP_pct_m35_w5_past', 'AccV_mean_past_20_maxs', 'fence_AccAP_pct_m15_w10_past', 'fence_AccAP_pct_m35_w10_past', 'AccML_min_past_20_maxs', 'precent_prograss_Time', 'AccV_mean_past_10_maxs', 'AccAP_std_past_20_mins', 'Visit', 'id', 'AccV_std_past_30_mins', 'AccV_std_past_40_mins', 'AccAP_max_past_30_maxs', 'AccAP_max_past_40_maxs', 'AccAP_min_past_30_mins', 'AccAP_min_past_40_mins', 'graph_AccAP_mean', 'graph_AccAP_std', 'graph_AccAP_min', 'graph_AccAP_max']
parms =   {'learning_rate': 0.03893406248134833, 'depth': 3, 'l2_leaf_reg': 0.07011275909861214}

de_train = pd.read_pickle("/kaggle/input/make-data-v9-de/train_de.pkl")[all_cols]
td_train = pd.read_pickle("/kaggle/input/make-data-v9-td/train_td.pkl")[all_cols]

de_valid = pd.read_pickle("/kaggle/input/make-data-v9-de/valid_de.pkl")[all_cols]
td_valid = pd.read_pickle("/kaggle/input/make-data-v9-td/valid_td.pkl")[all_cols]

td_train["Valid"] = 1
td_train["Task"] = 1

td_valid["Valid"] = 1
td_valid["Task"] = 1

df_for_cv =  de_train.append(td_train).reset_index(drop=True)
df_for_sub = df_for_cv.append(de_valid).append(td_valid).reset_index(drop=True)

# folds_train_indexes_cv = pickle.load(open("/kaggle/input/v9-make-folds/folds_train_indexs_cv.pkl","rb"))
# folds_train_indexes_full = pickle.load(open("/kaggle/input/v9-make-folds/folds_train_indexs_full.pkl","rb"))

# folds_valid_indexes_cv = pickle.load(open("/kaggle/input/v9-make-folds/folds_valid_indexs_cv.pkl","rb"))
# folds_valid_indexes_full = pickle.load(open("/kaggle/input/v9-make-folds/folds_valid_indexs_full.pkl","rb"))

In [3]:
# angry work-around
# def angry_workaround(df, indexs):
#     for fold in range(len(indexs)):
#         df.loc[indexs[fold][0], "target"] = 0    
#         df.loc[indexs[fold][1], "target"] = 1 
#         df.loc[indexs[fold][2], "target"] = 2    
#         df.loc[indexs[fold][3], "target"] = 3
#     return df


# df_for_cv = angry_workaround(df_for_cv, folds_train_indexes_cv)
# df_for_sub = angry_workaround(df_for_sub, folds_train_indexes_full)

In [4]:
# folds_train_indexs_full = pickle.load(open("/kaggle/input/v9-make-folds/folds_train_indexs_full.pkl", "rb"))
# train_ind = set([])
# for ele in folds_train_indexs_full:
#     for ind in ele:
#         train_ind.add(ind)
# relevant_ind = list(train_ind)

# df_for_sub = df_for_sub.loc[relevant_ind]

In [5]:
def get_model(parms= None, model_name="lgbm"):
    if model_name == "lgbm":
        if parms is None:
            model = lightgbm.LGBMClassifier()
        else:
            model = lightgbm.LGBMClassifier(**parms)
    elif model_name == "rf":
        if parms is None:
            model = RandomForestClassifier()
        else:
            model = RandomForestClassifier(**parms)     
            
    elif model_name == "xgboost":
        if parms is None:
            model = xgboost.XGBClassifier()
        else:
            model = xgboost.XGBClassifier(**parms) 
            
    elif model_name == "catboost":
        if parms is None:
            model = catboost.CatBoostClassifier()
        else:
            model = catboost.CatBoostClassifier(**parms) 
            
    elif model_name == "adaboost":
        if parms is None:
            model = AdaBoostClassifier()
        else:
            model = AdaBoostClassifier(**parms)
            
    return model

In [6]:
def fix_y(y):
    return y - 1 # (1,2,3) -> (0,1,2)

def fit_save(train_X,train_y, train_cols, cv_or_full,parms= None, model_name="lgbm"):
    no_train_cols = ["Time","Valid","Task",'Subject','file', 'id', 'target']    
    train_cols = [c for c in train_cols if c not in no_train_cols]
    
    model = get_model(parms, model_name)
    no_sh = False
    
    if set(train_y.unique()) == set([1,2,3]): # no SH
        train_y = fix_y(train_y)
        no_sh = True
    gc.collect()
    model.fit(train_X[train_cols], train_y)
    gc.collect()
    model_dict = {"model_name" : model_name,
                 "model": model,
                 "parms": parms,
                 "features": train_cols}
    pickle.dump(model_dict, open(f"model_dict_{model_name}_{cv_or_full}.pkl", "wb"))

In [7]:
#print(f"fit_save_{MODEL}_for_CV")
#fit_save(df_for_cv,df_for_cv["target"], train_cols=all_cols, parms= parms, model_name=MODEL, cv_or_full = "cv")

# print(f"Make_ENS_Xy_{MODEL}_for_CV")
# preds, gt = pipeline(df_for_cv, all_cols, folds=5, parms=parms, model_name=MODEL, cv_or_full = "cv")

In [8]:
print(f"fit_save_{MODEL}_for_submission")
fit_save(df_for_sub,df_for_sub["target"], train_cols=all_cols, parms= parms, model_name=MODEL, cv_or_full = "full")

# print(f"Make_ENS_Xy_{MODEL}_for_submission")
# preds, gt = pipeline(df_for_sub, all_cols, folds_total=folds_total, folds_start=folds_start,folds_end=folds_end,
#                      train_folds_list=folds_train_indexes_full , valid_folds_list= folds_valid_indexes_full, parms=parms, model_name=MODEL, cv_or_full = "full")


fit_save_catboost_for_submission
0:	learn: 1.3003826	total: 13.4s	remaining: 3h 43m 22s
1:	learn: 1.2257021	total: 25.5s	remaining: 3h 32m 24s
2:	learn: 1.1605847	total: 37.2s	remaining: 3h 25m 54s
3:	learn: 1.1026982	total: 51.9s	remaining: 3h 35m 25s
4:	learn: 1.0510254	total: 1m 3s	remaining: 3h 31m 57s
5:	learn: 1.0044480	total: 1m 15s	remaining: 3h 27m 56s
6:	learn: 0.9616575	total: 1m 27s	remaining: 3h 26m 21s
7:	learn: 0.9228552	total: 1m 38s	remaining: 3h 24m 30s
8:	learn: 0.8872830	total: 1m 50s	remaining: 3h 22m 51s
9:	learn: 0.8543165	total: 2m 3s	remaining: 3h 23m 6s
10:	learn: 0.8240295	total: 2m 15s	remaining: 3h 22m 21s
11:	learn: 0.7961629	total: 2m 27s	remaining: 3h 22m 6s
12:	learn: 0.7703267	total: 2m 40s	remaining: 3h 23m 5s
13:	learn: 0.7463508	total: 2m 53s	remaining: 3h 23m 6s
14:	learn: 0.7237263	total: 3m 5s	remaining: 3h 23m 33s
15:	learn: 0.7025790	total: 3m 18s	remaining: 3h 23m 36s
16:	learn: 0.6831439	total: 3m 30s	remaining: 3h 23m 20s
17:	learn: 0.664499